In [ ]:
# Attention-BLSTM classification example

# import the required modules
from __future__ import print_function
import pandas as pd
import numpy as np
import csv
import datetime

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)

import keras
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.optimizers import Adamax
from keras import initializers

from sklearn.metrics import confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize,LabelEncoder
from sklearn.utils import class_weight

# Attention implemented by https://github.com/CyberZHG/keras-self-attention
from keras_self_attention import SeqSelfAttention

# turn off the warnings, be careful when use this
#import warnings
#warnings.filterwarnings("ignore")

In [ ]:
# define variables
# parameters to be investigated in grid seearch
time_steps = [1,5,10]  # input history to include: [1,5,10]
lstm_sizes = [[16,8,4],[32,16,8],[64,32,16]] # number of neurons in the BLSTM layers: [[16,8,4],[32,16,8],[64,32,16]]
attention_widths = [1,2,4] # width of the local context for the attention layer: [1,2,4]

# other parameters
batch_size = 32 # for estimating error gradient
# number of total epochs to train the model
nb_epoch = 20
# optimization function
opt_func = Adamax(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08) 
# to prevent over-fitting
early_stopping = EarlyStopping(monitor='loss', patience=5)

In [ ]:
# reshape panda.DataFrame to Keras style: (batch_size, time_step, nb_features)
def reshape_data(data, n_prev):
    docX = []
    for i in range(len(data)):
        if i < (len(data)-n_prev):
            docX.append(data[i:i+n_prev])
        else: # the frames in the last window use the same context
            docX.append(data[(len(data)-n_prev):len(data)])
    alsX = np.array(docX)
    return alsX

# define the BLSTM model with attention
def attBLSTM(lstm_size, attention_width, nb_class, opt_func):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=lstm_size[0], return_sequences=True))) # BLSTM layer 1
    #model.add(RNN(LSTMCell(lstm_size[0]), return_sequences=True), kernel_initializer=initializers.glorot_uniform(seed=0))
    model.add(Bidirectional(LSTM(units=lstm_size[1], return_sequences=True))) # BLSTM layer 2
    #model.add(RNN(LSTMCell(lstm_size[1]), return_sequences=True), kernel_initializer=initializers.glorot_uniform(seed=0))
    model.add(Bidirectional(LSTM(units=lstm_size[2], return_sequences=True))) # BLSTM layer 3
    #model.add(RNN(LSTMCell(lstm_size[2]), return_sequences=True), kernel_initializer=initializers.glorot_uniform(seed=0))
    model.add(SeqSelfAttention(attention_width=attention_width, attention_activation='sigmoid')) # attention layer
    model.add(Dense(units=nb_class, activation='softmax')) # output layer, predict emotion dimensions seperately
    return model

# one-hot encoding of the class labels
def one_hot(labels, cat):
    labels_converted = []
    if cat == 'BASE':
        for label in labels:
            if label == 'STATIONARY':
                label_converted = [1,0,0,0]
            elif label == 'TO OPERATOR':
                label_converted = [0,1,0,0]
            elif label == 'ROTATING':
                label_converted = [0,0,1,0]
            elif label == 'TO PARTICIPANT':
                label_converted = [0,0,0,1]
            labels_converted.append(label_converted)
    elif cat == 'ARM':
        for label in labels:
            if label == 'STATIONARY':
                label_converted = [1,0,0]
            elif label == 'REACHING':
                label_converted = [0,1,0]
            elif label == 'TUCKING':
                label_converted = [0,0,1]
            labels_converted.append(label_converted)
    elif cat == 'OTP':
        for label in labels:
            if label == 'LEFT':
                label_converted = [1,0,0]
            elif label == 'MIDDLE':
                label_converted = [0,1,0]
            elif label == 'RIGHT':
                label_converted = [0,0,1]
            labels_converted.append(label_converted)
    labels_converted = np.asarray(labels_converted)
    return labels_converted

In [ ]:
# apply the best performing model
def model_eval(f_mode, c_mode, log_f, model, time_step):
    # X_trn = reshape_data(globals()['x_'+str(f_mode)+'_trn'], time_step)
    X_dev = reshape_data(globals()['x_'+str(f_mode)+'_dev'], time_step)
    X_tst = reshape_data(globals()['x_'+str(f_mode)+'_tst'], time_step)
    # Y_trn = reshape_data(globals()['y_'+str(c_mode)+'_trn'], time_step)
    Y_dev = reshape_data(globals()['y_'+str(c_mode)+'_dev'], time_step)
    Y_tst = reshape_data(globals()['y_'+str(c_mode)+'_tst'], time_step)

    # print results on training set
    # model.evaluate(X_trn, Y_trn, batch_size=batch_size)
    # trn_pred = model.predict(X_trn)
    # y_trn_non_category = [ np.argmax(t[0]) for t in Y_trn ]
    # y_trn_predict_non_category = [ np.argmax(t[0]) for t in trn_pred ]
    # print('Confusion Matrix on train set')
    # print(confusion_matrix(y_trn_non_category, y_trn_predict_non_category))
    # trn_f1 = f1_score(y_trn_non_category, y_trn_predict_non_category, average='weighted')
    # print('Weighted F1-score on train set:', trn_f1)
    # with open(log_f, 'a') as logfile:
        # logfile.write('Confusion Matrix on train set\n')
        # np.savetxt(logfile, confusion_matrix(y_trn_non_category, y_trn_predict_non_category))
        # logfile.write('Weighted F1-score on train set: %s' % trn_f1)

    # print results on dev set
    model.evaluate(X_dev, Y_dev, batch_size=batch_size)
    dev_pred = model.predict(X_dev)
    y_dev_non_category = [ np.argmax(t[0]) for t in Y_dev ]
    y_dev_predict_non_category = [ np.argmax(t[0]) for t in dev_pred ]
    print('Confusion Matrix on development set')
    print(confusion_matrix(y_dev_non_category, y_dev_predict_non_category))
    dev_f1 = f1_score(y_dev_non_category, y_dev_predict_non_category, average='weighted')
    print('Weighted F1-score on development set:', dev_f1)
    with open(log_f, 'a') as logfile:
        logfile.write('Confusion Matrix on development set\n')
        np.savetxt(logfile, confusion_matrix(y_dev_non_category, y_dev_predict_non_category))
        logfile.write('Weighted F1-score on development set: %s' % dev_f1)

    # print results on test set
    model.evaluate(X_tst, Y_tst, batch_size=batch_size)
    tst_pred = model.predict(X_tst)
    y_tst_non_category = [ np.argmax(t[0]) for t in Y_tst ]
    y_tst_predict_non_category = [ np.argmax(t[0]) for t in tst_pred ]
    print('Confusion Matrix on test set')
    print(confusion_matrix(y_tst_non_category, y_tst_predict_non_category))
    tst_f1 = f1_score(y_tst_non_category, y_tst_predict_non_category, average='weighted')
    print('Weighted F1-score on test set:', tst_f1)
    with open(log_f, 'a') as logfile:
        logfile.write('Confusion Matrix on test set\n')
        np.savetxt(logfile, confusion_matrix(y_tst_non_category, y_tst_predict_non_category))
        logfile.write('Weighted F1-score on test set: %s' % tst_f1)

In [ ]:
# data files
file_trn = 'data/ML/combined_ML_trn.csv'
file_dev = 'data/ML/combined_ML_dev.csv'
file_tst = 'data/ML/combined_ML_tst.csv'

# read in data
trn_data = pd.read_csv(file_trn, header=0)
dev_data = pd.read_csv(file_dev, header=0)
tst_data = pd.read_csv(file_tst, header=0)

# number of features
# nb_feat_time = 3 # ['time (s)', 'episode', 'step']
# nb_feat_kp_cor = 75 # (x,y,z) of the 25 facial and upper body keypoints
# nb_feat_kp_con = 25 # confidence of the facial and upper body keypoints
# nb_feat_task = 1 # task progress
# nb_feat_emo = 20 # categorical and arousal-valence for both cameras
# nb_feat_rw = 3 # reward values: operator's rating, emotional reward, combined
# nb_feat_all = nb_feat_time + nb_feat_kp_cor + nb_feat_kp_con + nb_feat_task + nb_feat_emo + nb_feat_rw
# number of classes
nb_class_base = 4 # {'STATIONARY', 'TO OPERATOR', 'ROTATING', 'TO PARTICIPANT'}
nb_class_arm = 3 # {'STATIONARY', 'REACHING', 'TUCKING'}
nb_class_otp = 3 # {'MIDDLE', 'LEFT', 'RIGHT'}

x_all_trn = trn_data.iloc[:,:127]
# drop S1 handover episodes in training set: episodes [0,1,2,3]
trn_no_S1_data = trn_data[trn_data.episode > 3]
X_no_S1_trn = trn_no_S1_data.iloc[:,:127]
# for ablation studies
x_no_time_trn = trn_data.iloc[:,3:127]
x_no_rw_trn = trn_data.iloc[:,3:124]
x_no_emo_trn = trn_data.iloc[:,23:124]
x_no_tp_trn = trn_data.iloc[:,23:123]
trn_data = trn_data[trn_data.columns.drop(list(trn_data.filter(regex='(confidence)')))]
x_no_conf_trn = trn_data.iloc[:,23:98]

x_all_dev = dev_data.iloc[:,:127]
# for ablation studies
x_no_time_dev = dev_data.iloc[:,3:127]
x_no_rw_dev = dev_data.iloc[:,3:124]
x_no_emo_dev = dev_data.iloc[:,23:124]
x_no_tp_dev = dev_data.iloc[:,23:123]
dev_data = dev_data[dev_data.columns.drop(list(dev_data.filter(regex='(confidence)')))]
x_no_conf_dev = dev_data.iloc[:,23:98]

x_all_tst = tst_data.iloc[:,:127]
# for ablation studies
x_no_time_tst = tst_data.iloc[:,3:127]
x_no_rw_tst = tst_data.iloc[:,3:124]
x_no_emo_tst = tst_data.iloc[:,23:124]
x_no_tp_tst = tst_data.iloc[:,23:123]
tst_data = tst_data[tst_data.columns.drop(list(tst_data.filter(regex='(confidence)')))]
x_no_conf_tst = tst_data.iloc[:,23:98]

In [ ]:
# one-hot encoding of the classes
y_base_trn = one_hot(trn_data['base status'], cat = 'BASE')
y_base_dev = one_hot(dev_data['base status'], cat = 'BASE')
y_base_tst = one_hot(tst_data['base status'], cat = 'BASE')
y_base_no_S1_trn = one_hot(trn_no_S1_data['base status'], cat = 'BASE')

y_arm_trn = one_hot(trn_data['arm status'], cat = 'ARM')
y_arm_dev = one_hot(dev_data['arm status'], cat = 'ARM')
y_arm_tst = one_hot(tst_data['arm status'], cat = 'ARM')
y_arm_no_S1_trn = one_hot(trn_no_S1_data['arm status'], cat = 'ARM')

y_otp_trn = one_hot(trn_data['handover status'], cat = 'OTP')
y_otp_dev = one_hot(dev_data['handover status'], cat = 'OTP')
y_otp_tst = one_hot(tst_data['handover status'], cat = 'OTP')
y_otp_no_S1_trn = one_hot(trn_no_S1_data['handover status'], cat = 'OTP')

In [ ]:
# Mode control for feature set and class
time_stamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
c_mode_list = ['base', 'arm', 'otp']

f_mode = 'all'
c_mode = c_mode_list[1]

In [ ]:
# output files
file_out = 'exp1/logs/LSTM_' + f_mode + '_' + c_mode + '_sum_' + time_stamp + '.txt'
file_log = 'exp1/logs/LSTM_' + f_mode + '_' + c_mode + '_log_' + time_stamp + '.txt'
file_pred = 'exp1/logs/LSTM_' + f_mode + '_' + c_mode + '_pred_' + time_stamp + '.txt'
model_dir = 'exp1/models/LSTM/' + c_mode + '/' + f_mode + '/'

In [ ]:
para_list = []
tst_pred_list = []
f1_list = []
count = 1

training = False

# Grid search on dev set
if training:
    for time_step in time_steps:
        # pad data
        X_trn = reshape_data(globals()['x_'+str(f_mode)+'_trn'], time_step)
        X_dev = reshape_data(globals()['x_'+str(f_mode)+'_dev'], time_step)
        Y_trn = reshape_data(globals()['y_'+str(c_mode)+'_trn'], time_step)
        Y_dev = reshape_data(globals()['y_'+str(c_mode)+'_dev'], time_step)
        for lstm_size in lstm_sizes:
            for attention_width in attention_widths:
                para_list.append([time_step, lstm_size, attention_width]) # save parameter set
                print('\n================================ No. %s of 27 ========================================' % count)
                print('\nParameters: time_step = %s, [h1, h2, h3] = %s, attention_width = %s\n' 
                      % (time_step, lstm_size, attention_width))
                # build model with given parameters
                model = attBLSTM(lstm_size, attention_width, globals()['nb_class_'+str(c_mode)], opt_func)
                # compile the model
                model.compile(loss='categorical_crossentropy', optimizer=opt_func, metrics=['categorical_accuracy'])
                # training the model
                model.fit(X_trn, Y_trn, batch_size=batch_size, epochs=nb_epoch, 
                          validation_split=0.05, callbacks=[early_stopping], verbose=2)
                # evaluation
                model.evaluate(X_dev, Y_dev, batch_size=batch_size)
                # save model
                model_f = model_dir + str(count)
                model.save(model_f)

                # save predictions
                tst_pred = model.predict(X_dev)
                tst_pred_list.append(tst_pred) # save predictions

                # print confusion matrix
                y_test_non_category = [ np.argmax(t[0]) for t in Y_dev ]
                y_predict_non_category = [ np.argmax(t[0]) for t in tst_pred ]
                print('Confusion Matrix on dev set')
                print(confusion_matrix(y_test_non_category, y_predict_non_category))
                tst_f1 = f1_score(y_test_non_category, y_predict_non_category, average='weighted')
                f1_list.append(tst_f1) # save f1 score
                print('Weighted F1-score on dev set:', tst_f1)
                # print grid search log
                with open(file_log, 'a') as logfile:
                    logfile.write('\n================================ No. %s of 27 ========================================\n' % count)
                    logfile.write('F1 = %s; Parameters: time_step = %s, [h1, h2, h3] = %s, attention_width = %s\n' 
                                  % (tst_f1, time_step, lstm_size, attention_width))
                    logfile.write('Confusion Matrix on dev set\n')
                    np.savetxt(logfile, confusion_matrix(y_test_non_category, y_predict_non_category))          
                count = count + 1

In [ ]:
# get the best parameter set
best = f1_list.index(max(f1_list)) # find the highest F1 score
best_count = best + 1
result = f1_list[best]
para = para_list[best]
# prediction = tst_pred_list[best]
print('Best Run at No.%s; F1 = %s; Parameters: time_step = %s, [h1,h2,h3] = %s, attention_width = %s\n' 
      % (best_count, result, para[0], para[1], para[2]))

with open(file_out, 'a') as outfile:
    outfile.write('Best Run at No.%s; F1 = %s; Parameters: time_step = %s, [h1,h2,h3] = %s, attention_width = %s\n' 
                   % (best_count, result, para[0], para[1], para[2]))

save predictions in case of significance test
with open(file_pred, 'a') as predfile:
    for pred in prediction:
        indi_pred = []
        indi_pred = pred[0] # reform the seq prediction to individual samples
        row = ', '.join(map(str, indi_pred))
        predfile.write('%s\n' % row)

# apply on train, dev, and test sets
best_model_f = model_dir + str(best_count)
best_model = keras.models.load_model(best_model_f)
model_eval(f_mode, c_mode, file_out, best_model, para[0])

In [ ]:
# ablation study with the best model parameters
f_ab_list = ['all', 'no_time', 'no_rw', 'no_emo', 'no_tp', 'no_conf']
nb_epoch_ab = 20

for f_ab in f_ab_list:
    # output files
    ab_file_out = 'exp2/logs/LSTM_' + f_ab + '_' + c_mode + '_sum_' + time_stamp + '.txt'
    ab_model_dir = 'exp2/models/LSTM/' + c_mode + '/' + f_ab + '/'
    with open(ab_file_out, 'a') as outfile:
        outfile.write('Parameters: features = %s, class = %s, time_step = %s, [h1,h2,h3] = %s, attention_width = %s\n' 
                   % (f_ab, c_mode, para[0], para[1], para[2]))
    print('Parameters: features = %s, class = %s, time_step = %s, [h1,h2,h3] = %s, attention_width = %s\n' 
          % (f_ab, c_mode, para[0], para[1], para[2]))
    X_trn = reshape_data(globals()['x_'+str(f_mode)+'_trn'], para[0])
    X_dev = reshape_data(globals()['x_'+str(f_mode)+'_dev'], para[0])
    X_tst = reshape_data(globals()['x_'+str(f_mode)+'_tst'], para[0])
    Y_trn = reshape_data(globals()['y_'+str(c_mode)+'_trn'], para[0])
    Y_dev = reshape_data(globals()['y_'+str(c_mode)+'_dev'], para[0])
    Y_tst = reshape_data(globals()['y_'+str(c_mode)+'_tst'], para[0])
    
    model = attBLSTM(para[1], para[2], globals()['nb_class_'+str(c_mode)], opt_func)
    model.compile(loss='categorical_crossentropy', optimizer=opt_func, metrics=['categorical_accuracy'])
    model.fit(X_trn, Y_trn, batch_size=batch_size, epochs=nb_epoch_ab, 
              validation_split=0.05, callbacks=[early_stopping], verbose=2)
    
    model_eval(f_ab, c_mode, ab_file_out, model, para[0])
    ab_model_f = ab_model_dir
    model.save(ab_model_f)

In [ ]:
# training on s2 and s3 data only

# Mode control for feature set and class
time_stamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
nb_epoch_no_S1 = 20

# output files
no_S1_file_out = 'exp4/logs/LSTM_no_S1_' + c_mode + '_sum_' + time_stamp + '.txt'
no_S1_model_dir = 'exp4/models/LSTM/' + c_mode + '/no_S1/'
with open(no_S1_file_out, 'a') as outfile:
    outfile.write('Parameters: features = no_S1, class = %s, time_step = %s, [h1,h2,h3] = %s, attention_width = %s\n' 
               % (c_mode, para[0], para[1], para[2]))
print('Parameters: features = no_S1, class = %s, time_step = %s, [h1,h2,h3] = %s, attention_width = %s\n' 
      % (c_mode, para[0], para[1], para[2]))
X_trn = reshape_data(X_no_S1_trn, para[0])
X_dev = reshape_data(x_all_dev, para[0])
X_tst = reshape_data(x_all_tst, para[0])
Y_trn = reshape_data(globals()['y_'+str(c_mode)+'_no_S1_trn'], para[0])
Y_dev = reshape_data(globals()['y_'+str(c_mode)+'_dev'], para[0])
Y_tst = reshape_data(globals()['y_'+str(c_mode)+'_tst'], para[0])

model = attBLSTM(para[1], para[2], globals()['nb_class_'+str(c_mode)], opt_func)
model.compile(loss='categorical_crossentropy', optimizer=opt_func, metrics=['categorical_accuracy'])
model.fit(X_trn, Y_trn, batch_size=batch_size, epochs=nb_epoch_no_S1, 
          validation_split=0.05, callbacks=[early_stopping], verbose=2)

# print results on training set
# model.evaluate(X_trn, Y_trn, batch_size=batch_size)
# trn_pred = model.predict(X_trn)
# y_trn_non_category = [ np.argmax(t[0]) for t in Y_trn ]
# y_trn_predict_non_category = [ np.argmax(t[0]) for t in trn_pred ]
# print('Confusion Matrix on train set')
# print(confusion_matrix(y_trn_non_category, y_trn_predict_non_category))
# trn_f1 = f1_score(y_trn_non_category, y_trn_predict_non_category, average='weighted')
# print('Weighted F1-score on train set:', trn_f1)
# with open(ab_file_out, 'a') as logfile:
    # logfile.write('Confusion Matrix on train set\n')
    # np.savetxt(logfile, confusion_matrix(y_trn_non_category, y_trn_predict_non_category))
    # logfile.write('Weighted F1-score on train set: %s' % trn_f1)

# print results on dev set
model.evaluate(X_dev, Y_dev, batch_size=batch_size)
dev_pred = model.predict(X_dev)
y_dev_non_category = [ np.argmax(t[0]) for t in Y_dev ]
y_dev_predict_non_category = [ np.argmax(t[0]) for t in dev_pred ]
print('Confusion Matrix on development set')
print(confusion_matrix(y_dev_non_category, y_dev_predict_non_category))
dev_f1 = f1_score(y_dev_non_category, y_dev_predict_non_category, average='weighted')
print('Weighted F1-score on development set: %s' % dev_f1)
with open(no_S1_file_out, 'a') as logfile:
    logfile.write('Confusion Matrix on development set\n')
    np.savetxt(logfile, confusion_matrix(y_dev_non_category, y_dev_predict_non_category)) 
    logfile.write('Weighted F1-score on development set: %s' % dev_f1)


# print results on test set
model.evaluate(X_tst, Y_tst, batch_size=batch_size)
tst_pred = model.predict(X_tst)
y_tst_non_category = [ np.argmax(t[0]) for t in Y_tst ]
y_tst_predict_non_category = [ np.argmax(t[0]) for t in tst_pred ]
print('Confusion Matrix on test set')
print(confusion_matrix(y_tst_non_category, y_tst_predict_non_category))
tst_f1 = f1_score(y_tst_non_category, y_tst_predict_non_category, average='weighted')
print('Weighted F1-score on test set:', tst_f1)
with open(no_S1_file_out, 'a') as logfile:
    logfile.write('Confusion Matrix on test set\n')
    np.savetxt(logfile, confusion_matrix(y_tst_non_category, y_tst_predict_non_category)) 
    logfile.write('Weighted F1-score on test set: %s' % tst_f1)

#model_eval(f_ab, c_mode, ab_file_out, model, para[0])
no_S1_model_f = no_S1_model_dir
model.save(no_S1_model_f)